# 注意力分数（Attention Scores）

注意力分数（Attention Scores）是注意力机制（Attention Mechanism）中的一个关键概念。它们用于衡量输入序列中每个元素对当前生成步骤的重要性。通过计算注意力分数，模型可以动态地关注输入序列的不同部分，从而提高生成结果的质量。

## 1. 基本概念

### 1.1 背景

在传统的序列到序列（Seq2Seq）模型中，编码器将整个输入序列压缩成一个固定大小的上下文向量（Context Vector），然后解码器根据这个向量生成输出序列。然而，这种压缩可能导致信息的丢失，尤其是在处理长序列时。

### 1.2 注意力机制

注意力机制通过在解码器的每一步生成过程中，动态地计算输入序列中每个元素的重要性（注意力权重），并根据这些权重对输入序列进行加权求和，得到一个加权上下文向量。这个加权上下文向量被用来生成当前的输出元素。

### 1.3 注意力分数

注意力分数是计算注意力权重的基础。它们表示解码器当前的隐藏状态（查询）与编码器的每个隐藏状态（键）之间的相似度。注意力分数越高，表示对应的输入元素对当前生成步骤越重要。

f(\mathbf{q}, (\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)) = \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i \in \mathbb{R}^v,

正如上图所示，选择不同的注意力评分函数a会导致不同的注意力汇聚操作。 

![ATF](https://zh-v2.d2l.ai/_images/attention-output.svg "ATF")

## 2. 计算注意力分数

注意力分数的计算方法有多种，常见的方法包括点积（Dot Product）、缩放点积（Scaled Dot Product）、加性（Additive）或拼接（Concatenation）等。

### 2.1 点积（Dot Product）

点积是最简单的计算方法之一。它通过计算查询和键之间的点积来得到注意力分数。

- **公式**：
  \[
  \text{score}(Q, K) = Q \cdot K
  \]
  其中，\( Q \) 是查询向量，\( K \) 是键向量。

### 2.2 缩放点积（Scaled Dot Product）

缩放点积在点积的基础上进行了缩放，以避免点积结果过大或过小。缩放因子通常是键向量的维度 \( d_k \) 的平方根。

- **公式**：
  \[
  \text{score}(Q, K) = \frac{Q \cdot K}{\sqrt{d_k}}
  \]
  其中，\( d_k \) 是键向量的维度。

### 2.3 加性（Additive）或拼接（Concatenation）

加性或拼接方法通过将查询和键拼接或相加，然后通过一个前馈神经网络（Feedforward Neural Network）来计算注意力分数。

- **公式**：
  \[
  \text{score}(Q, K) = \text{FFN}(\text{concat}(Q, K))
  \]
  其中，FFN 是一个前馈神经网络。

## 3. 归一化

计算得到注意力分数后，通常需要对其进行归一化处理，以得到注意力权重。归一化方法通常使用 Softmax 函数，将注意力分数转换为概率分布。

- **公式**：
  \[
  \alpha_i = \text{softmax}(\text{score}(Q, K_i))
  \]
  其中，\( \alpha_i \) 是第 \( i \) 个输入元素的注意力权重。

## 4. 加权求和

根据归一化后的注意力权重，对编码器的隐藏状态（值）进行加权求和，得到加权上下文向量。

- **公式**：
  \[
  \text{context} = \sum_i \alpha_i \cdot V_i
  \]
  其中，\( V_i \) 是第 \( i \) 个输入元素的值向量。

## 5. 应用场景

- **机器翻译**：在生成翻译后的句子时，注意力分数可以帮助模型关注源语言句子中的重要部分。
- **文本摘要**：在生成摘要时，注意力分数可以帮助模型关注原文中的关键信息。
- **问答系统**：在生成回答时，注意力分数可以帮助模型关注问题中的关键部分和相关上下文。

## 6. 优点与局限性

### 6.1 优点

- **动态关注**：注意力分数允许模型在每一步生成过程中动态地关注输入序列的不同部分，从而更好地捕捉输入序列中的重要信息。
- **处理长序列**：相比于传统的Seq2Seq模型，注意力机制在处理长序列时表现更好，因为它不需要将整个输入序列压缩成一个固定大小的向量。

### 6.2 局限性

- **计算复杂度**：注意力分数的计算复杂度较高，尤其是在处理长序列时。
- **可解释性**：虽然注意力分数提高了模型的性能，但它也使得模型的可解释性降低，因为注意力分数是动态计算的，难以直观理解。

## 7. 总结

注意力分数是注意力机制中的一个关键概念，用于衡量输入序列中每个元素对当前生成步骤的重要性。通过计算注意力分数，模型可以动态地关注输入序列的不同部分，从而提高生成结果的质量。尽管计算复杂度较高，但注意力分数在许多NLP任务中表现出色，成为现代深度学习模型的核心组件之一。

## 简单代码实现

### 掩蔽softmax操作

In [ ]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

### 加性注意力

a(\mathbf q, \mathbf k) = \mathbf w_v^\top \text{tanh}(\mathbf W_q\mathbf q + \mathbf W_k \mathbf k) \in \mathbb{R},

In [ ]:
#@save
class AdditiveAttention(nn.Module):
    """加性注意力"""
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hidden)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return torch.bmm(self.dropout(self.attention_weights), values)

### 缩放点积注意力

a(\mathbf q, \mathbf k) = \mathbf{q}^\top \mathbf{k}  /\sqrt{d}.

In [ ]:
#@save
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)